# Exploratory Data Analysis and Visualization using Altair

"""
This notebook explores water quality sensor readings collected from various locations. 
We'll utilize Python libraries like Pandas and Altair to analyze the dataset and create 
interactive visualizations, addressing the following key questions:

1. **Trend Analysis:**
    - Identify temporal trends in chemical contamination levels.
    - Analyze spatial variations in contamination across sensor sites.

2. **Anomaly Detection:**
    - Detect sudden changes in contamination levels over time.
    - Identify sensor sites with significantly different contamination levels compared to others.

3. **Data Quality Assessment:**
    - Identify and handle missing data.
    - Analyze the consistency of data collection frequency.
    - Detect and address unrealistic values.
"""

Prepare & load data

In [88]:
#Setup

import pandas as pd
import altair as alt
from vega_datasets import data
from ipywidgets import interact, Dropdown
import altair_viewer

import warnings

alt.data_transformers.enable('default', max_rows=None) #to disable the 5000 rows limit
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
# Enable Altair's default data transformations
#To disable the 5000 rows limit
alt.data_transformers.enable('default', max_rows=None) 

In [50]:
#Loading the data
df = pd.read_csv("C:\\Users\\Rishabh Mehta\\Downloads\\cst4060\\Coursework 2 - Altair\\Boonsong Lekagul waterways readings.csv")

Data Cleaning and Preprocessing

In [51]:
# Rename columns for consistency
df.columns = df.columns.str.strip().str.replace(' ', '_').str.lower() 

# Rename measures for consistency
df['measure'] = df['measure'].str.strip().str.replace(' ', '_')

In [ ]:
#Display dataset information
df.head()

In [ ]:
df.info()

In [ ]:
# Statistical summary
df.describe()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
# Check for missing data
missing_summary = df.isnull().sum()
print("Missing Data Summary:\n", missing_summary)

In [ ]:
# Check for duplicates and unrealistic values
print(f"Duplicate rows: {df.duplicated().sum()}")
unrealistic_values = df[df['value'] < 0]
print("Unrealistic Values:", unrealistic_values)

Exploring unique values in key columns

In [ ]:
df['location'].unique()

In [ ]:
df['location'].value_counts()

In [ ]:
df['measure'].unique()

In [ ]:
df['measure'].value_counts()

In [ ]:
len(df['measure'].unique())

In [64]:
#Convert 'sample_date' to datetime format
df['sample_date'] = pd.to_datetime(df['sample_date'], format='%d-%b-%y') 

In [ ]:
# Extract date-related information
df['year'] = df['sample_date'].dt.year
df['month'] = df['sample_date'].dt.month_name()
df['year_month'] = df['sample_date'].dt.strftime('%Y-%m')
df['week_day'] = df['sample_date'].dt.day_name()  # Monday=0, Sunday=6

# Display dataset information
df.info()

In [ ]:
df.head()   

## Part 1: Trend Analysis Visualizations

In [ ]:
# Visualization: Total Count by Locations
visual_1 = alt.Chart(df).mark_bar().encode(
    x=alt.X('location:N', title="Locations", axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('count(value):Q', title="Total Count of Measures"),
    color=alt.Color('location:N', legend=alt.Legend(title="Locations")),  # Color by location
    tooltip=[
        alt.Tooltip('location:N', title="Location"),
        alt.Tooltip('count()', title="Total Count", format=",")
    ]
).properties(
    width=700,
    height=400,
    title=alt.TitleParams(
        text="Total Count by Locations",
        subtitle="Color-coded by Locations",
        fontSize=16,
        anchor='start'
    )
)
visual_1

The chart above shows the total count of data collected at each location. It appears that locations like Boonsri, Chai, and Sakda have significantly higher data collection counts compared to other locations. This helps identify which locations have the highest or lowest number of sensor recordings, which is relevant for understanding spatial variations in contamination levels.

In [ ]:
# Visualization: Total Count by Measures
visual_2 = alt.Chart(df).mark_rect().encode(
    x=alt.X('count(value):Q', title="Total Count"),
    y=alt.Y('measure:N', title="Measures", sort='-x'),
    color=alt.Color('count(value):Q', title="Count"),
    tooltip=[
        alt.Tooltip('measure:N', title="Measure"),
        alt.Tooltip('count(value):Q', title="Total Count", format=",")
    ]
).properties(
    width=700,
    height=2000,  # Adjust height to fit all measures or scroll
    title=alt.TitleParams(
        text="Total Count by Measures",
        fontSize=16,
        anchor='start'
    )
)
visual_2

The chart above provides a clear overview of the data collection frequency for different water quality measures. Measures like Water_Temperature, Nitrites, Ammonium, and Nitrates have the highest number of data points, indicating they are measured more frequently than others.The color gradient effectively represents the variation in data counts. Measures with higher counts are depicted in darker shades, while those with lower counts appear lighter.  It helps visualize which chemicals have been measured the most and gives an insight into the data density for each chemical.

In [ ]:
# Counting how many days of data we have for each measure
days_per_measure = df.groupby('measure')['sample_date'].nunique()

# Calculating total number of days in the dataset
total_days = df['sample_date'].nunique()

# Calculating the percentage of days with data for each measure
data_availability = (days_per_measure / total_days * 100).reset_index()
data_availability.columns = ['measure', 'availability']

# Sorting measures by availability in descending order
data_availability = data_availability.sort_values('availability', ascending=True)

# Counting measures with at least 75% data availability
usable_measures = len(data_availability[data_availability['availability'] >= 75])

# Create the bar chart
chart = alt.Chart(data_availability).mark_bar().encode(
    y=alt.Y('measure:N', sort='-x', title='Measure'),
    x=alt.X('availability:Q', title='Data Availability (%)'),
    color=alt.condition(
        alt.datum.availability >= 75,
        alt.value('steelblue'),
        alt.value('lightgray')
    ),
    tooltip=['measure', alt.Tooltip('availability:Q', format='.1f')]
).properties(
    title=f'Data Availability by Measure (Total days: {total_days}, Usable measures: {usable_measures})',
    width=600,
    height=2000
)

# Add a rule to show the 75% threshold
rule = alt.Chart(pd.DataFrame({'threshold': [75]})).mark_rule(color='red').encode(
    x='threshold:Q'
)

# Combine the chart and the rule
visual_3 = (chart + rule).interactive()

# Display the chart
visual_3


Water_temprature, Nitrates, Ammonium, Nitrites, Total_phosphorus, Orthophosphate-phosphorus, Dissolved_oxygen and Biochemical_Oxygen are the measures with data availability over 75% threshold. 

In [ ]:
# Counting how many days of data we have for each measure
days_per_measure = df.groupby('measure')['sample_date'].nunique()

# Calculating total number of days in the dataset
total_days = df['sample_date'].nunique()

# Calculating the percentage of days with data for each measure
data_availability = (days_per_measure / total_days) * 100

# Sorting measures by availability in descending order
data_availability_sorted = data_availability.sort_values(ascending=True)

# Counting measures with at least 75% data availability (less than 25% unavailable)
usable_measures = len(data_availability[data_availability >= 75])

# Print results
print(f"\nTotal number of days in dataset: {total_days}")
print(f"Number of usable measures (≥75% data available): {usable_measures}")

# Print availability for each measure
print("\nData availability for each measure:")
for measure in data_availability.index:
    print(f"Measure {measure}: {data_availability[measure]:.1f}% available")

In [ ]:
# Categorizing Contaminants - Feature Engineering
contamination_mapping = {
    "Biological Pollutants": [
        "Macrozoobenthos", "Total_coliforms", "Fecal_coliforms", "Fecal_streptococci"
    ],
    "Nutrient Pollution": [
        "Dissolved_oxygen", "Ammonium", "Nitrites", "Nitrates", "Orthophosphate-phosphorus",
        "Total_phosphorus", "Total_nitrogen", "Inorganic_nitrogen", "Organic_nitrogen",
        "Total_dissolved_phosphorus"
    ],
    "Chemical Pollutants": [
        "Chlorides", "Sulphates", "Bicarbonates", "Carbonates", "Silica_(SiO2)",
        "Total_hardness", "Total_dissolved_salts", "Cyanides", "Sulfides",
        "Total_organic_carbon", "Dissolved_organic_carbon", "Dissolved_silicates",
        "AOX", "Atrazine", "p,p-DDT", "gamma-Hexachlorocyclohexane", "Anionic_active_surfactants",
        "Heptachloroepoxide", "Heptachlor", "Endosulfan_(alpha)", "Endosulfan_(beta)",
        "p,p-DDD", "p,p-DDE", "alpha-Hexachlorocyclohexane", "beta-Hexaxchlorocyclohexane",
        "Aldrin", "Dieldrin", "Endrin", "Methoxychlor", "Simazine", "Metolachlor",
        "Alachlor", "Tetrachloromethane", "1,2,4-Trichlorobenzene", "1,2,3-Trichlorobenzene",
        "Pentachlorobenzene", "Acenaphthene", "Acenaphthylene", "Anthracene",
        "Benzo(a)anthracene", "Chrysene", "Naphthalene", "Phenanthrene", "Pyrene",
        "Hexachlorobenzene", "Isodrin", "PCB_28", "PCB_52", "PCB_101", "PCB_118",
        "PCB_138", "PCB_153", "PCB_180", "Trifluralin", "Fluoranthene", "Fluorene",
        "Indeno(1,2,3-c,d)pyrene", "Benzo(a)pyrene", "Benzo(g,h,i)perylene",
        "Benzo(b)fluoranthene", "Benzo(k)fluoranthene", "Selenium", "AGOC-3A",
        "Methylosmoline", "Chlorodinine", "PAHs"
    ],
    "Sediment Pollution": ["Total_extractable_matter"],
    "Thermal Pollution": ["Water_temperature", "Oxygen_saturation"],
    "Oil and Grease": ["Petroleum_hydrocarbons"],
    "Radioactive Substances": ["Cesium"],
    "Groundwater Contaminants": ["Barium", "Berilium", "Boron"],
    "Metals/Inorganic Contaminants": [
        "Sodium", "Potassium", "Calcium", "Magnesium", "Iron", "Manganese",
        "Zinc", "Copper", "Chromium", "Lead", "Cadmium", "Mercury", "Nickel",
        "Arsenic", "Aluminium"
    ],
    "Organic Pollution Indicators": [
        "Biochemical_Oxygen", "Chemical_Oxygen_Demand_(Cr)", "Chemical_Oxygen_Demand_(Mn)"
    ]
}

# Reverse mapping for efficient lookups
measure_to_category = {
    measure: category
    for category, measures in contamination_mapping.items()
    for measure in measures
}

# Mapping
df['contamination_category'] = df['measure'].map(measure_to_category)

# Display the DataFrame with contamination categories
print(df)

In [ ]:
# Calculate the number of measures per category
category_summary = df['contamination_category'].value_counts().reset_index()
category_summary.columns = ['contamination_category', 'count']

# Sort by count in ascending order
category_summary = category_summary.sort_values('count', ascending=True)

# Create a horizontal bar chart for category counts
visual_4 = alt.Chart(category_summary).mark_bar().encode(
    x=alt.X('count:Q', title='Measure Count'),
    y=alt.Y('contamination_category:N', title='Contamination Category', sort='x'),
    color=alt.Color('count:Q', scale=alt.Scale(scheme='tealblues'), title='Count'),
    tooltip=[
        alt.Tooltip('contamination_category:N', title='Category'),
        alt.Tooltip('count:Q', title='Count')
    ]
).properties(
    title='Distribution of Measures by Contamination Category',
    width=600,
    height=400
).configure_axis(
    labelAngle=0
)

# Display the updated visualization
visual_4


The chart above provides a clear overview of the distribution of water quality measures across different contamination categories. It shows that Chemical Pollutants and Metals/Inorganic Contaminants have the highest number of measures, indicating a greater focus on monitoring these categories. On the other hand, categories like Radioactive Substances and Oil and Grease have very few associated measures count.

In [ ]:
# Calculate daily frequency
daily_frequency = df.groupby('sample_date').size().reset_index(name='count')

# Rename columns to avoid issues with special characters or reserved names
daily_frequency.rename(columns={'sample_date': 'date'}, inplace=True)

# Create a base chart for daily frequency
base = alt.Chart(daily_frequency).encode(
    x=alt.X('date:T', title='Date'),
    y=alt.Y('count:Q', title='Record Count'),
    tooltip=['date:T', 'count:Q']
).properties(
    title='Daily Data Collection Frequency',
    width=900,
    height=300
)

# Create a bar chart
bar_chart = base.mark_bar()

# Create a trend line using a linear regression
trend_line = base.transform_regression('date', 'count', method='poly', order = 2).mark_line(color='red')

# Combine the bar chart and trend line
visual_5 = bar_chart + trend_line

# Show the chart
visual_5

The trend line shows a downward trajectory from 62 record count in Jan 1998 to 13 at the end of Dec 2016. 

In [ ]:
# Aggregate the data by month to calculate the total count of records
monthly_count = df.groupby(df['sample_date'].dt.to_period('M')).size().reset_index(name='count')
monthly_count.columns = ['month', 'count']

# Convert month to datetime for Altair compatibility
monthly_count['month'] = monthly_count['month'].dt.to_timestamp()

# Base chart for total count of values input during the month
month_collection = alt.Chart(monthly_count).mark_line(interpolate='step-after').encode(
    x=alt.X('month:T', title='Month'),
    y=alt.Y('count:Q', title='Total Count of Records'),
    tooltip=[
        alt.Tooltip('month:T', title='Month'),
        alt.Tooltip('count:Q', title='Total Count')
    ]
).properties(
    title='Monthly Data Collection Count with Polynomial Trend Line',
    width=600,
    height=300
)

# Polynomial regression line of degree 2
polynomial_trend_monthly = alt.Chart(monthly_count).transform_regression(
    'month', 'count', method='poly', order=2
).mark_line(color='orange').encode(
    x='month:T',
    y='count:Q'
)

# Combine the base chart and the polynomial trend line
visual_6 = month_collection + polynomial_trend_monthly

# Display the chart
visual_6


Periods of higher data collection (visible as taller spikes in the chart) might correspond to specific events, such as changes in environmental monitoring protocols, intensified data collection campaigns, or abnormal environmental events like chemical contamination in the above chart.

The polynomial curve shows an overall increase in data collection from 1998 to approximately 2010. This could indicate an expansion in monitoring activities, more sensors deployed, or an increased focus on gathering data. 

After the peak around 2010, the curve begins to flatten or show a decline in data collection, which may suggest a decrease in monitoring intensity or a reduction in the number of sensors or measurements being recorded in later years. This decline could point to resource constraints, changes in monitoring priorities, or external factors impacting data collection efforts.

In [ ]:
# Aggregate the data by year to calculate the total count of records
yearly_count = df.groupby(df['sample_date'].dt.year).size().reset_index(name='count')
yearly_count.columns = ['year', 'count']

# Convert year to datetime for Altair compatibility
yearly_count['year'] = pd.to_datetime(yearly_count['year'], format='%Y')

# Base chart for total count of values input during the year
year_collection = alt.Chart(yearly_count).mark_line(interpolate='step-after').encode(
    x=alt.X('year:T', title='Year'),
    y=alt.Y('count:Q', title='Total Count of Records'),
    tooltip=[
        alt.Tooltip('year:T', title='Year'),
        alt.Tooltip('count:Q', title='Total Count')
    ]
).properties(
    title='Yearly Data Collection Count with Polynomial Trend Line',
    width=600,
    height=300
)

# Polynomial regression line of degree 2
polynomial_trend = alt.Chart(yearly_count).transform_regression(
    'year', 'count', method='poly', order=2
).mark_line(color='orange').encode(
    x='year:T',
    y='count:Q'
)

# Combine the base chart and the polynomial trend line
visual_7 = year_collection + polynomial_trend

# Display the chart
visual_7


The data appears to be highly variable, with significant peaks and valleys at different points in time.

Periods of higher data collection (visible as taller spikes in the chart) might correspond to specific events, such as changes in environmental monitoring protocols, intensified data collection campaigns, or abnormal environmental events like chemical contamination.

Lower collection periods could indicate gaps in data collection, possibly due to operational issues or logistical constraints, which may be worth investigating further.

The peaks and valleys in the blue step line suggest seasonal variability in data collection. This could mean that monitoring efforts are concentrated during certain months or seasons, reflecting seasonal changes in environmental factors that might require more frequent data collection. 

 The initial increase in data collection indicates a ramping up of monitoring efforts, which is a positive sign for understanding environmental conditions in the preserve.

 The drop in data collection starting around 2010 could signal a need for improved monitoring. This decline could be linked to reduced funding, limited resources, or a focus on different priorities that result in less frequent data collection.

In [ ]:
# Aggregate data to calculate the yearly collection frequency
yearly_data = df.groupby(['location', 'year']).size().reset_index(name='record_count')

# Create a bar chart to show yearly collection frequency
visual_8 = alt.Chart(yearly_data).mark_bar().encode(
    x=alt.X('year:O', title='Year', axis=alt.Axis(labelAngle=90)),
    y=alt.Y('record_count:Q', title='Collection Frequency'),
    color=alt.Color('record_count:Q', title='Frequency', scale=alt.Scale(scheme='greens')),
    facet=alt.Facet('location:N', columns=3, title='Sensor Locations'),
    tooltip=[
        alt.Tooltip('location:N', title='Location'),
        alt.Tooltip('year:O', title='Year'),
        alt.Tooltip('record_count:Q', title='Frequency')
    ]
).properties(
    width=200,
    height=150,
    title='Yearly Collection Frequency of Sensor Recordings by Location'
).interactive()

# Display the modified visualization
visual_8


A significant concentration of data collected between 2003 and 2012, particularly for the locations Boonsri, Chai, Kannika, and Sakda. Therefore, we have chosen to focus our analysis on this subset of the dataset to ensure a robust and meaningful exploration. Locations with a sharp drop in data collection could suggest malfunctioning sensors or external factors affecting the frequency of sampling, potentially missing crucial data about environmental contamination.

In [ ]:
# Aggregate data by location and day of the week
weekday_data = df.groupby(['location', 'week_day']).size().reset_index(name='record_count')

# Ensure the days are in the correct order (Monday to Sunday)
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekday_data['week_day'] = pd.Categorical(weekday_data['week_day'], categories=weekday_order, ordered=True)

# Create the visualization
visual_9 = alt.Chart(weekday_data).mark_bar().encode(
    x=alt.X('week_day:N', title='Day of the Week', sort=weekday_order),
    y=alt.Y('record_count:Q', title='Collection Frequency'),
    color=alt.Color('record_count:Q', title='Frequency', scale=alt.Scale(scheme='purples')),
    facet=alt.Facet('location:N', columns=3, title='Sensor Locations'),
    tooltip=[
        alt.Tooltip('location:N', title='Location'),
        alt.Tooltip('week_day:N', title='Day of the Week'),
        alt.Tooltip('record_count:Q', title='Frequency')
    ]
).properties(
    width=200,
    height=150,
    title='Daily Collection Frequency of Sensor Recordings by Location'
).interactive()

# Display the visualization
visual_9


It is useful for understanding if certain days are underrepresented or overrepresented in terms of data collection, revealing potential issues with data collection consistency. Anomalies here would suggest specific times where sensor data may be sparse, leading to potential gaps in understanding chemical trends that may impact the bird population

## Part 2: Anomaly Detection Visualizations

In [ ]:
# Function to identify outliers using Z-Score
def identify_outliers_zscore(data, threshold=3):
    mean = data.mean()
    std = data.std()
    z_scores = (data - mean) / std
    return abs(z_scores) > threshold

# Add an outlier flag to the dataset
df['outlier'] = df.groupby(['measure', 'year'])['value'].transform(identify_outliers_zscore)

# Aggregate data for trend analysis
trend_data = (
    df.groupby(['year', 'measure', 'outlier'])
    .agg(total_value=('value', 'sum'), average_value=('value', 'mean'), count=('value', 'size'))
    .reset_index()
)

# Scatter plot to show values with outlier highlighting
scatter_chart = alt.Chart(trend_data).mark_point(filled=True).encode(
    x=alt.X('year:O', title='Year'),
    y=alt.Y('average_value:Q', title='Average Value'),
    color=alt.condition(
        alt.datum.outlier,
        alt.value('red'),  # Highlight outliers in red
        alt.value('steelblue')  # Normal data in blue
    ),
    size=alt.Size('count:Q', title='Record Count', scale=alt.Scale(range=[10, 200])),
    tooltip=[
        alt.Tooltip('measure:N', title='Measure'),
        alt.Tooltip('year:O', title='Year'),
        alt.Tooltip('average_value:Q', title='Avg Value', format='.2f'),
        alt.Tooltip('total_value:Q', title='Total Value', format=',.2f'),
        alt.Tooltip('count:Q', title='Count'),
        alt.Tooltip('outlier:Q', title='Outlier Flag')
    ]
).properties(
    title='Scatter Plot with Outlier Highlighting',
    width=900,
    height=600
)

# Add regression lines for trend analysis
trend_lines = alt.Chart(trend_data[~trend_data['outlier']]).transform_regression(
    'year', 'average_value', groupby=['measure']
).mark_line(color='orange').encode(
    x='year:O',
    y='average_value:Q',
    color='measure:N'
)

# Combine scatter plot and trend lines
visual_10 = scatter_chart + trend_lines

# Display the chart
visual_10


 Outliers are detected using the Z-score method with a threshold of 3. Data points with Z-scores greater than 3 or less than -3 are flagged as outliers.

 Outliers in Red: These are values that are significantly higher or lower than the average for a particular measure and year. For example, if a measure like Nitrates shows unusually high values in certain years, those data points are flagged as outliers in red.

 Outliers can indicate significant environmental changes or events that may require closer examination. For example, a spike in Chemical Oxygen Demand (COD) in a particular year could signal a contamination event, industrial spill, or other factors that dramatically increased the chemical levels.

 The size of the points indicates which measures have more data available, suggesting more reliable trend analysis for those chemicals.

In [ ]:
# Function to identify outliers using Z-Score on counts
def identify_outliers_zscore(data, threshold=3):
    mean = data.mean()  # Calculate the mean of the counts
    std = data.std()    # Calculate the standard deviation of the counts
    z_scores = (data - mean) / std  # Z-score calculation for count values
    return abs(z_scores) > threshold  # Flag counts that are outliers

# Aggregate data for trend analysis (count of values)
trend_data = (
    df.groupby(['year', 'measure'])
    .agg(count=('value', 'size'))  # Counting the number of values for each group
    .reset_index()
)

# Identify outliers based on the count of values using Z-score
trend_data['outlier'] = identify_outliers_zscore(trend_data['count'])

# Filter out the outliers to only include non-outliers
non_outliers_data = trend_data[trend_data['outlier'] == False]

# Scatter plot for non-outliers with color based on 'measure'
visual_11 = alt.Chart(non_outliers_data).mark_point(filled=True).encode(
    x=alt.X('year:O', title='Year'),
    y=alt.Y('count:Q', title='Count of Value'),
    color='measure:N',  # Color by 'measure' (categorical)
    size=alt.Size('count:Q', title='Record Count', scale=alt.Scale(range=[10, 200])),
    tooltip=[
        alt.Tooltip('measure:N', title='Measure'),
        alt.Tooltip('year:O', title='Year'),
        alt.Tooltip('count:Q', title='Count of Value'),
        alt.Tooltip('outlier:Q', title='Outlier')  # Tooltip will show if outlier
    ]
).properties(
    title='Non-Outliers Only (Colored by Measure)',
    width=900,
    height=600
)

# Display the chart with only non-outliers
visual_11


The outliers have been removed, as the chart only displays the non-outlier data. 

Outliers, which are identified based on the Z-score method (threshold = 3), are points that deviate significantly from the average (either exceptionally high or low). 

By removing them, the plot focuses on the more regular data trends, providing a clearer view of the data behavior without the influence of extreme values.

Measures with a steady count of records are well-monitored, which is important for maintaining an accurate and up-to-date picture of contamination levels.

Measures with fewer data points (smaller scatter plot points) may need more attention. It’s important to ensure that all critical environmental indicators are consistently monitored to detect trends and potential risks.

Variations in the size of points and year-to-year changes highlight the need for ongoing, adaptive monitoring practices to respond to environmental shifts or emerging issues.



In [ ]:
# Filter the dataset for measures and years
selected_measures = [
    'Water_temperature', 'Nitrates', 'Ammonium', 'Nitrites',
    'Total_phosphorus', 'Orthophosphate-phosphorus',
    'Dissolved_oxygen', 'Biochemical_Oxygen'
]

filtered_df = df[
    (df['measure'].isin(selected_measures)) & 
    (df['year'] >= 2003)
].copy()

# Create a simpler selection
selector = alt.selection_single(
    fields=['measure'],
    bind={"measure": alt.binding_select(options=selected_measures)},
    name="Select"
)

# Create the basic chart
chart = alt.Chart(filtered_df).mark_circle(
    size=60,
    opacity=0.6
).encode(
    x='value:Q',
    y='location:N',
    color='location:N',
    tooltip=['location:N', 'value:Q', 'year:Q']
).properties(
    width=600,
    height=400,
    title='Water Quality Analysis by Location'
).add_selection(
    selector
).transform_filter(
    selector
)

# Add mean lines
mean_lines = alt.Chart(filtered_df).mark_rule(
    color='white'
).encode(
    x='mean(value):Q',
    y='location:N'
).transform_filter(
    selector
)

# Combine the charts
visual_12 = chart + mean_lines

visual_12

Water_temperature distributions hint at the influence of local geography and possibly shading, with some locations showing greater thermal stability than others.

Nitrogen compounds (Nitrates, Nitrites, and Ammonium) show interesting location-specific patterns. While some sampling points maintain consistent levels, others display considerable fluctuations, potentially indicating localized sources of nutrient input or varying rates of biological processing.

Phosphorus measurements (Total_phosphorus and Orthophosphate-phosphorus) demonstrate notable spatial heterogeneity, suggesting different levels of agricultural influence or wastewater input across the sampling network.

Dissolved_oxygen and Biochemical_Oxygen reveal insights about ecosystem health. The varying ranges across locations could indicate differences in organic matter load, aquatic vegetation, or water flow characteristics.

In [ ]:
# Define the measures we want to analyze
selected_measures = [
    'Water_temperature', 'Nitrates', 'Ammonium', 'Nitrites',
    'Total_phosphorus', 'Orthophosphate-phosphorus',
    'Dissolved_oxygen', 'Biochemical_Oxygen'
]

# Filter the dataset for only the selected measures and data from 2003 onwards
subset_data = df[(df['measure'].isin(selected_measures)) & (df['year'] >= 2003)]

# List to store individual box plots
box_plots = []

# Loop through each measure and create a box plot
for measure in selected_measures:
    # Filter data for the specific measure
    measure_data = subset_data[subset_data['measure'] == measure]
    
    # Create a box plot for the current measure
    box_plot = alt.Chart(measure_data).mark_boxplot().encode(
        x=alt.X('value:Q', title='Value Range'),
        y=alt.Y('location:N', title=f'{measure} Locations'),
        color=alt.Color('location:N', legend=None, scale=alt.Scale(scheme='viridis')),
        tooltip=[
            alt.Tooltip('location:N', title='Location'),
            alt.Tooltip('value:Q', title='Value')
        ]
    ).properties(
        width=700,
        height=150,
        title=f'{measure} Distribution Across Locations'
    )
    
    # Add the box plot to the list
    box_plots.append(box_plot)

# Combine all the box plots into a single vertical layout
visual_13 = alt.vconcat(*box_plots).resolve_scale(color='independent')

# Display the final visualization
visual_13


The Box represents the interquartile range (IQR), where the middle 50% of the data falls. The top and bottom edges of the box are the 75th and 25th percentiles respectively.

The box plot for Water Temperature shows a wide range of values for different locations, with some locations exhibiting high variance in the data. Some locations have a more concentrated distribution, while others show more spread out data.

There are several outliers, which could indicate exceptional temperature events, potentially linked to environmental anomalies or specific monitoring periods.

The Nitrates distribution shows a fairly wide spread, with values ranging from low to high concentrations. Some locations have values around 0, while others show much higher concentrations.

A few outliers indicate high nitrate concentrations in certain locations, which could be associated with pollution or runoff events. These outliers are significant and should be further investigated.

The distribution for Ammonium shows similar trends to nitrates, but with a slightly smaller range in some locations. The spread in the data suggests that Ammonium levels are generally low, with occasional higher values in specific locations.

The plot shows some high Ammonium values in specific locations, possibly indicating contamination spikes.

The Nitrites distribution is more compressed compared to other measures, with most data points concentrated in a smaller range. This indicates relatively stable nitrite levels across locations.

There are fewer outliers compared to other measures, suggesting more consistent monitoring of nitrites.

The Total Phosphorus distribution shows moderate variation in values across locations. There are no extreme values, but some locations have higher concentrations, which could be related to pollution sources like agricultural runoff.

There are a few outliers, particularly in the higher value range, which could be indicative of pollution or specific environmental events.

Orthophosphate Phosphorus shows a distribution similar to Total Phosphorus, with some locations having higher concentrations. These elevated values could be due to localized sources of phosphorus.

Outliers appear mostly in the higher concentration range, suggesting possible pollution events that need to be further examined.

The Dissolved Oxygen distribution shows a wider spread, with some locations having relatively high or low oxygen levels. This is important for understanding the overall health of the water.

Outliers for Dissolved Oxygen indicate extreme variations, which could be caused by ecological changes, such as algae blooms or seasonal changes.

 The Biochemical Oxygen Demand (BOD) distribution shows a large spread, with a few locations having significantly higher concentrations. Higher BOD levels are typically associated with organic pollution.

 Several outliers are visible, particularly in the higher range, which could indicate contamination events or areas with poor water quality.

## Part 3: Data Quality Analysis

In [ ]:
# Aggregate the count of recordings for each measure over the years
agg_dataset_new = df.groupby(['measure', 'year']).size().reset_index(name='record_count')

# Create a heatmap instead of a bar chart
visual_14 = alt.Chart(agg_dataset_new).mark_rect().encode(
    x=alt.X('year:O', title='Year'),
    y=alt.Y('measure:N', title='Measure', sort='-x'),
    color=alt.Color(
        'record_count:Q',
        title='Record Count',
        scale=alt.Scale(scheme='greens')
    ),
    tooltip=[
        alt.Tooltip('year:O', title='Year'),
        alt.Tooltip('measure:N', title='Measure'),
        alt.Tooltip('record_count:Q', title='Record Count')
    ]
).properties(
    title='Yearly Recording Frequency by Measure',
    width=900,
    height=1200
).configure_axis(
    labelAngle=0
)

# Display the visualization
visual_14


Measures like "Nitrates", "Water Temperature", "Ammonium", and "Dissolved Oxygen" consistently show higher data collection frequencies across the years (darker green areas). These chemicals are likely to be of particular interest for regular environmental monitoring, possibly due to their relevance to water quality or ecosystem health.

The consistency in recording for these measures suggests that they are prioritized in monitoring efforts, possibly because they are critical to assessing the overall environmental status.

Some measures, such as "Petroleum Hydrocarbons" or "Arsenic", show lighter shades and appear to have sparse or inconsistent data collection. These measures may have been less frequently monitored or were only recorded during certain years.

 The inconsistent monitoring of these measures could indicate that they are either not as high of a priority for monitoring or that data collection is more sporadic due to logistical constraints. This could also suggest areas where more consistent monitoring is needed.

 For some measures, such as "Dissolved Oxygen" or "Biochemical Oxygen Demand", there are clear peaks in data collection around specific years (e.g., 2003-2008), where the color intensity is darker.

 These peaks could indicate intensive monitoring efforts or environmental concerns during those years, such as increased pollution or regulatory measures requiring more frequent measurements. Investigating these periods further could reveal important insights into the environmental conditions during these years.

 Some years show largely lighter shades across most measures, indicating that data collection was sparse for those years. These periods might reflect resource limitations, changes in monitoring priorities, or gaps in data collection.

 Missing data in key years can limit the ability to track long-term environmental trends accurately. It may be beneficial to investigate the causes of these gaps to improve future monitoring practices.

 

In [ ]:
pivot_data = df.pivot_table(index='year', columns='location', values='value', aggfunc='count').reset_index()
completeness = pivot_data.count(axis=0).sort_values(ascending=False)
sorted_measures = completeness.index.tolist()
sorted_measures.remove('year')

heatmap = alt.Chart(pivot_data).mark_rect().encode(
    y=alt.Y('location:N', title=None, sort=sorted_measures),
    x=alt.X('year:O', title='Year'),
    color=alt.Color('value:Q', scale=alt.Scale(scheme='inferno'), title='Count Value'),
    tooltip=[alt.Tooltip('location:N'), alt.Tooltip('year:O'), alt.Tooltip('value:Q')]
).properties(
    width=800,
    height=400,
    title='Heatmap of Count of Recordings for Each Location Over the Years'
).transform_fold(
    fold=[col for col in pivot_data.columns if col != 'year'],
    as_=['location', 'value']
)

visual_15 = heatmap
visual_15

Boonsri and Chai show significant spikes in data collection, particularly around the years 2005 to 2009, where the color intensity is at its highest (yellow and orange).

These locations likely experienced intensive monitoring efforts during these years, possibly due to increased environmental concerns, regulation changes, or specific contamination events that triggered more frequent data collection.

Tansanee, Achara and Decha show minimal data collection throughout the years with no data recorded prior to 2009, represented by darker colors (purple). This suggests that these locations were either under-monitored or did not require frequent data collection during the years shown.

These locations may need more focused monitoring in the future, especially if they are critical areas for understanding environmental changes or risks.

Certain years, such as 2005, 2007, and 2010, show more intensive data collection across various locations, suggesting that these years had special monitoring programs or environmental events that prompted more frequent data collection.

These years could correlate with specific events (e.g., environmental disasters, new regulations, or heightened pollution risks) that required increased monitoring. Further investigation into the causes of these spikes could provide valuable context.



In [ ]:
# Select key indicators
key_indicators = ['Dissolved_oxygen', 'Biochemical_Oxygen','Nitrates','Nitrates', 'Ammonium', 'Total_phosphorus','Orthophosphate-phosphorus', 'Water_temperature']

# Filter data for key indicators
filtered_df = df[df['measure'].isin(key_indicators)]

# Create measure selection
measure_dropdown = alt.binding_select(options=key_indicators, name='Select Measure')
measure_select = alt.selection_point(fields=['measure'], bind=measure_dropdown)

# Create base chart
base_chart = alt.Chart(filtered_df).mark_line(point=True).encode(
    x='sample_date:T',
    y=alt.Y('value:Q', title='Concentration'),
    color='location:N',
    tooltip=['location', 'measure', 'value', 'sample_date']
).properties(
    width=250,
    height=200
).add_params(
    measure_select
).transform_filter(
    measure_select
)

# Create a chart for each location
charts = []
locations = filtered_df['location'].unique()
for location in locations:
    chart = base_chart.transform_filter(
        alt.datum.location == location
    ).properties(
        title=f'{location} - Water Quality Measure'
    )
    charts.append(chart)

# Combine charts side by side
visual_16 = alt.concat(*charts, spacing=5).resolve_scale(color='independent')

visual_16


Each location has its own chart, allowing you to examine how the concentration of each water quality measure changes over time.

For instance, a location like Boonsri might show consistent Dissolved Oxygen levels, indicating a healthy aquatic environment, while another location might show fluctuating levels of Nitrates, suggesting potential pollution issues.

By examining the line charts over time, you can identify seasonal patterns or long-term trends in each water quality measure. 

For example, you might notice that Water Temperature rises in summer months or that Ammonium concentrations spike after rainfall, indicating runoff from agricultural fields.

The side-by-side layout makes it easy to compare trends across locations. You might notice that certain locations, such as Achara, show relatively stable conditions across all measures, while others like Decha or Kohsoom exhibit more dramatic fluctuations, possibly indicating environmental challenges specific to those locations.



In [ ]:
# Ensure year_month column is in datetime format
df['year_month'] = pd.to_datetime(df['year_month'])

# Time-series plot for each measure
visual_17 = alt.Chart(df).mark_line().encode(
    x=alt.X('year_month:T', title='Year-Month'),  # Time type for proper time scaling
    y=alt.Y('count(value):Q', title='Count of Value'),
    color=alt.Color('measure:N', title='Measure'),  # Color for differentiation
    tooltip=[
        alt.Tooltip('measure:N', title='Measure'),
        alt.Tooltip('count(value):Q', title='Count of Value'),
        alt.Tooltip('year_month:T', title='Year-Month')
    ]
).properties(
    title='Time-Series of Measures',
    width=250,  # Adjusted for facets
    height=200
).facet(
    column=alt.Column('measure:N', title='Measure')  # Facet by measure
).resolve_scale(
    y='independent'  # Allow independent y-scales for each measure
)

# Display the chart
visual_17


The plot is faceted by measure, meaning that each environmental measure (e.g., Nitrates, Ammonium, Dissolved Oxygen) has its own individual time-series plot.

This allows easy comparison of trends across measures, as each plot focuses on a specific measure’s data, making it easier to track how each parameter changes over time.

The time-series plots show the temporal trends of each environmental measure. 

For example, we see seasonal variations in Water Temperature, with higher values in the warmer months, or fluctuations in Nitrates tied to agricultural runoff during certain periods.

Spikes or Drops in data for certain months or years could indicate environmental events (e.g., industrial discharge, heavy rainfall, or regulatory changes) that triggered more intensive monitoring or resulted in unusual levels of certain chemicals.

The faceted layout allows us to compare trends for each measure side by side. For example, you may observe that Dissolved Oxygen and Biochemical Oxygen Demand might show complementary trends (e.g., a decrease in oxygen levels correlating with a rise in BOD).

Measures like Nitrates and Ammonium might show spikes at the same times, potentially linked to nutrient pollution events. Conversely, Water Temperature might follow seasonal patterns with higher values in the summer months and lower values in the winter months.

The count of values for each measure can give insights into the frequency of data collection for each parameter. If a measure shows low counts for certain months, it could indicate gaps in monitoring or data collection irregularities.

Measures with higher counts might be more consistently monitored, while those with fewer data points could reflect less frequent measurements or possibly limited environmental concerns for that measure during specific periods.

Outliers or sudden spikes in the data can highlight significant environmental events. For example, an unusual spike in Nitrates in a specific year could be tied to increased agricultural runoff, while a spike in Ammonium could indicate an industrial discharge event.

These anomalies can prompt further investigation into potential causes, such as pollution sources or specific ecological changes, and inform policy and management decisions.

